<a href="https://colab.research.google.com/github/OswaldoLopezAlcaraz/TI_1_Practica3/blob/main/TI_1_Practica3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica 3

**Nombre:** Oswaldo López Alcaraz

**e-mail:** oswaldo.lopez5103@alumnos.udg.mx

## MODULES


In [1]:
import numpy as np
import pandas as pd
import math
from scipy.stats import wrapcauchy

import plotly.graph_objects as go

from scipy.spatial import distance

## CLASSES


In [2]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y

    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)

    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)

    # rotate vector
    def rotated(self, angle):
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

## FUNCTIONS

In [11]:

def bm_2d(n_steps=1000, speed=6, s_pos=[0,0]):
  """
  Summary:
    This function generates a Brownian Walk.
  Arguments:
    n_steps: number of steeps that comprises the entire walk.
    speed: number of units per steps.
    s_pos: coordinates (x and y ) where the walk starts.
  Returns:
    BM_2d_df: dataframe with the walk in form of coordinates.
  """
  # Init velocity vector
  velocity = Vec2d(speed, 0)

  BM_2d_df = pd.DataFrame(columns = ['x_pos','y_pos'])
  temp_df = pd.DataFrame([{'x_pos':s_pos[0], 'y_pos':s_pos[1]}])

  BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  for i in range(n_steps-1):
    turn_angle = np.random.uniform(low=-np.pi, high=np.pi)
    velocity = velocity.rotated(turn_angle)

    temp_df = pd.DataFrame([{'x_pos':BM_2d_df.x_pos[i]+velocity.x, 'y_pos':BM_2d_df.y_pos[i]+velocity.y}])
    BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  return BM_2d_df



def crw_2d(n_steps=1000, speed=6, s_pos=[0,0], crw_exp=0.3):
  """
  Summary:
    This function generates a Correlated Random Walk.
  Arguments:
    n_steps: number of steeps that comprises the entire walk
    speed: number of units per steps
    s_pos: coordinates (x and y ) where the walk starts.
    crw_exp:  exponent used in the Cauchy distribution
  Returns:
    CRW_2d_df: dataframe with the walk in form of coordinates.
  """
  # Init of velocity vector
  velocity = Vec2d(speed,0)

  CRW_2d_df = pd.DataFrame( columns= ['x_pos','y_pos'])
  temp_df = pd.DataFrame([{'x_pos':s_pos[0],'y_pos': s_pos[1]},])
  CRW_2d_df = pd.concat([CRW_2d_df, temp_df], ignore_index=True)

  # Brownian walker in motion and assignation of directions
  for i in range(1, n_steps):

    # Determination of angle using cauchy distribution
    turn_angle = wrapcauchy.rvs(crw_exp)
    velocity = velocity.rotated(turn_angle)

    # Motion of Brownian walker
    temp_df = pd.DataFrame([{'x_pos':CRW_2d_df.x_pos[i-1]+velocity.x,
                           'y_pos':CRW_2d_df.y_pos[i-1]+velocity.y,}])
    CRW_2d_df = pd.concat([CRW_2d_df,temp_df], ignore_index=True)

  return CRW_2d_df



def my_distance_function(origin, destination):
  """
  Arguments:
    origin: x & y coordinates that represent the origin of a trajectory
    destination: x & y coordinate representing the destination of a trajectory
  Returns:
    Euclidiean distance between two points in the form of int or float.
  """
  return math.sqrt(((destination.x_pos - origin.x_pos)**2) +
                    (( destination.y_pos - origin.y_pos)**2) )


def smd(motion_df):
  """
  Arguments:
    motion_df: dataframe with coordinates that represents a certain type of displacement.
  Returns:
    dataframe of length motion.df.shape[0] - 1 containing SMD in for each step
  """
  smd = pd.DataFrame(columns=['MSD'])

  for n in range(1,motion_df.shape[0]):

    displacement_vec = np.array([(my_distance_function(motion_df.iloc[i-n], motion_df.iloc[i]))**2
                              for i in range(n, motion_df.shape[0],1)])
    tmp_df = pd.DataFrame([{'MSD':np.mean(displacement_vec)}])
    smd = pd.concat([smd,tmp_df], ignore_index=True)

  return smd

## BROWNIAN WALK

In [ ]:
n_steps = 1000
s_pos = [0,0]
speed = 6

# Init velocity vector
velocity = Vec2d(speed, 0)

BM_2d_df = pd.DataFrame(columns = ['x_pos','y_pos'])
temp_df = pd.DataFrame([{'x_pos':s_pos[0], 'y_pos':s_pos[1]}])

BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

for i in range(n_steps-1):
  turn_angle = np.random.uniform(low=-np.pi, high=np.pi)
  velocity = velocity.rotated(turn_angle)

  temp_df = pd.DataFrame([{'x_pos':BM_2d_df.x_pos[i]+velocity.x, 'y_pos':BM_2d_df.y_pos[i]+velocity.y}])
  BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)



In [ ]:
BM_2d_df = bm_2d(2000, 3, [5,7])

# Init figure
fig_BM_2d = go.Figure()

# Plot trajectory
fig_BM_2d.add_trace(go.Scatter(
    x = BM_2d_df.x_pos,
    y = BM_2d_df.y_pos,
    marker = dict(size=2),
    line = dict(width=1),
    mode='lines',
    name='BM_2d_trajectory',
    showlegend=True
))

fig_BM_2d.show()

## CORRELATED RANDOM WALK

In [ ]:
# Init variables
n_steps = 1000
s_pos = [0,0]
speed = 5
times = np.linspace(0,1,n_steps)
crw_exp = 0.28

# Init of velocity vector
velocity = Vec2d(speed,0)

CRW_2d_df = pd.DataFrame( columns= ['x_pos','y_pos'])
temp_df = pd.DataFrame([{'x_pos':s_pos[0],'y_pos': s_pos[1]},])
CRW_2d_df = pd.concat([CRW_2d_df, temp_df], ignore_index=True)


# Brownian walker in motion and assignation of directions
for i in range(1, n_steps):

  # Determination of angle using cauchy distribution
  turn_angle = wrapcauchy.rvs(crw_exp)
  velocity = velocity.rotated(turn_angle)

  # Motion of Brownian walker
  temp_df = pd.DataFrame([{'x_pos':CRW_2d_df.x_pos[i-1]+velocity.x,
                           'y_pos':CRW_2d_df.y_pos[i-1]+velocity.y,}])
  CRW_2d_df = pd.concat([CRW_2d_df,temp_df], ignore_index=True)


In [ ]:
crw_2d(100, 3 , [0,0], 0.3)

# 2D plotting
fig_BM_pdf = go.Figure()

fig_BM_pdf.add_trace(go.Scatter(x = CRW_2d_df.x_pos,
                                y = CRW_2d_df.y_pos,
                                marker = dict(size=2),
                                line = dict(width=1),
                                mode = 'lines',
                                name = 'CB_Walker',
                                showlegend = True))

fig_BM_pdf.show()

## METRICS

# Actividad 1: Path-Length - (BM1 vs BM2 vs CRW)

In [ ]:
BM_2d_df_1 = bm_2d(1000, 1, [0,0])
BM_2d_df_2 = bm_2d(1000, 2, [0,0])
CRW_2d_df_n = crw_2d(1000, 3, [0,0], 0.3)

#CRW_2d_df_6_9 = pd.read_csv('trajectories/crw_6_9.csv')

In [ ]:
dis_BM_1 = np.array([my_distance_function(BM_2d_df_1.iloc[i-1],BM_2d_df_1.iloc[i]) for i in range(1,BM_2d_df_1.shape[0])])
pl_BM_1 = np.cumsum(dis_BM_1)

dis_BM_2 = np.array([my_distance_function(BM_2d_df_2.iloc[i-1],BM_2d_df_2.iloc[i]) for i in range(1,BM_2d_df_2.shape[0])])
pl_BM_2 = np.cumsum(dis_BM_2)

dis_CRW_n = np.array([my_distance_function(CRW_2d_df_n.iloc[i-1],CRW_2d_df_n.iloc[i]) for i in range(1,CRW_2d_df_n.shape[0])])
pl_CRW_n = np.cumsum(dis_CRW_n)


In [ ]:
# Init figure
fig_path_length = go.Figure()

# Plot trajectory
fig_path_length.add_trace(go.Scatter(
    x = np.arange(len(pl_BM_1))+1,
    y = pl_BM_1,
    marker = dict(size=2),
    line = dict(width=2),
    mode='lines',
    name='path_length_BM_1',
    showlegend=True
))

fig_path_length.add_trace(go.Scatter(
    x = np.arange(len(pl_BM_2))+1,
    y = pl_BM_2,
    marker = dict(size=2),
    line = dict(width=6),
    mode='lines',
    name='path_length_BM_2',
    showlegend=True
))

fig_path_length.add_trace(go.Scatter(
    x = np.arange(len(pl_CRW_n))+1,
    y = pl_CRW_n,
    marker = dict(size=2),
    line = dict(width=2),
    mode='lines',
    name='path_length_CRW',
    showlegend=True
))

fig_path_length.show()

## Actividad 2: Mean Squared Displacement - (BM vs CRW)

In [ ]:
#bm_2d_6 = bm_2d(1000, 2, [0,0])
#crw_2d_6 = crw_2d(1000, 6, [0,0], crw_exp=0.9)
bm_2d_6 = pd.read_csv('trajectories/brownian_6.csv')
crw_2d_6 = pd.read_csv('trajectories/crw_6_9.csv')



bm_smd = smd(bm_2d_6)
crw_smd = smd(crw_2d_6)
bm_smd.max()

In [ ]:


fig_path_smd = go.Figure()

# Plot smd
fig_path_smd.add_trace(go.Scatter(
    x = bm_smd,
    y = np.arange(0,bm_smd.max()),
    marker = dict(size=2),
    line = dict(width=2),
    mode='lines',
    name='bm_smd',
    showlegend=True
))

fig_path_smd.add_trace(go.Scatter(
    x = np.arange(0,crw_2d_6.shape[0]),
    y = crw_smd,
    marker = dict(size=2),
    line = dict(width=2),
    mode='lines',
    name='crw_smd',
    showlegend=True
))
fig_path_smd.show()